In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-27 06:31:43--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.5’

postgresql-42.2.16. 100%[===================>] 979.38K  5.59MB/s    in 0.2s    

2021-12-27 06:31:44 (5.59 MB/s) - ‘postgresql-42.2.16.jar.5’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Crime").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://datacrime-bucket.s3.us-east-2.amazonaws.com/Texas_2020.csv"
spark.sparkContext.addFile(url)
texas_2020_df = spark.read.csv(SparkFiles.get("Texas_2020.csv"), sep=",", header=True, inferSchema=True)

In [5]:
# Show DataFrame
texas_2020_df.show()

+---------+-----------+-------------+-------------+-----------------+----------+---------------+--------------------+----------------+----------+------------------+---------------------+-----------+------------+--------------+-------------+---------------------+-------------+
|AGENCY_ID|INCIDENT_ID|INCIDENT_DATE|INCIDENT_HOUR|CLEARED_EXCEPT_ID|OFFENSE_ID|OFFENSE_TYPE_ID|     PUB_AGENCY_NAME|AGENCY_TYPE_NAME|POPULATION|SUBURBAN_AREA_FLAG|POPULATION_GROUP_CODE|COUNTY_NAME|OFFENSE_CODE|  OFFENSE_NAME|CRIME_AGAINST|OFFENSE_CATEGORY_NAME|OFFENSE_GROUP|
+---------+-----------+-------------+-------------+-----------------+----------+---------------+--------------------+----------------+----------+------------------+---------------------+-----------+------------+--------------+-------------+---------------------+-------------+
|    29534|  133835511|     8-Dec-20|           11|                5| 161190322|             51|Independent Schoo...|           Other|         0|                 Y|     

In [6]:
url ="https://datacrime-bucket.s3.us-east-2.amazonaws.com/Texas_2019.csv"
spark.sparkContext.addFile(url)
texas_2019_df = spark.read.csv(SparkFiles.get("Texas_2019.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
texas_2019_df.show()

+---------+-----------+-------------+-------------+-----------------+----------+---------------+---------------+----------------+----------+------------------+---------------------+------------+------------+--------------------+-------------+---------------------+-------------+
|AGENCY_ID|INCIDENT_ID|INCIDENT_DATE|INCIDENT_HOUR|CLEARED_EXCEPT_ID|OFFENSE_ID|OFFENSE_TYPE_ID|PUB_AGENCY_NAME|AGENCY_TYPE_NAME|POPULATION|SUBURBAN_AREA_FLAG|POPULATION_GROUP_CODE| COUNTY_NAME|OFFENSE_CODE|        OFFENSE_NAME|CRIME_AGAINST|OFFENSE_CATEGORY_NAME|OFFENSE_GROUP|
+---------+-----------+-------------+-------------+-----------------+----------+---------------+---------------+----------------+----------+------------------+---------------------+------------+------------+--------------------+-------------+---------------------+-------------+
|    19658|  111280682|    02-JAN-19|          8.0|                6| 136090264|              5|         Sinton|            City|      5373|                 Y|    

In [7]:
url ="https://datacrime-bucket.s3.us-east-2.amazonaws.com/Texas_2018.csv"
spark.sparkContext.addFile(url)
texas_2018_df = spark.read.csv(SparkFiles.get("Texas_2018.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
texas_2018_df.show()

+---------+-----------+-------------+-------------+-----------------+----------+---------------+---------------+----------------+----------+------------------+---------------------+--------------------+------------+--------------------+-------------+---------------------+-------------+
|AGENCY_ID|INCIDENT_ID|INCIDENT_DATE|INCIDENT_HOUR|CLEARED_EXCEPT_ID|OFFENSE_ID|OFFENSE_TYPE_ID|PUB_AGENCY_NAME|AGENCY_TYPE_NAME|POPULATION|SUBURBAN_AREA_FLAG|POPULATION_GROUP_CODE|         COUNTY_NAME|OFFENSE_CODE|        OFFENSE_NAME|CRIME_AGAINST|OFFENSE_CATEGORY_NAME|OFFENSE_GROUP|
+---------+-----------+-------------+-------------+-----------------+----------+---------------+---------------+----------------+----------+------------------+---------------------+--------------------+------------+--------------------+-------------+---------------------+-------------+
|    19869|  111183863|    22-DEC-18|         12.0|                6| 135975594|             49|         Dallas|            City|   1362465

In [8]:
url ="https://datacrime-bucket.s3.us-east-2.amazonaws.com/Texas_2017.csv"
spark.sparkContext.addFile(url)
texas_2017_df = spark.read.csv(SparkFiles.get("Texas_2017.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
texas_2017_df.show()

+---------+-----------+-------------+-------------+-----------------+----------+---------------+---------------+----------------+----------+------------------+---------------------+-------------+------------+--------------------+-------------+---------------------+-------------+
|AGENCY_ID|INCIDENT_ID|INCIDENT_DATE|INCIDENT_HOUR|CLEARED_EXCEPT_ID|OFFENSE_ID|OFFENSE_TYPE_ID|PUB_AGENCY_NAME|AGENCY_TYPE_NAME|POPULATION|SUBURBAN_AREA_FLAG|POPULATION_GROUP_CODE|  COUNTY_NAME|OFFENSE_CODE|        OFFENSE_NAME|CRIME_AGAINST|OFFENSE_CATEGORY_NAME|OFFENSE_GROUP|
+---------+-----------+-------------+-------------+-----------------+----------+---------------+---------------+----------------+----------+------------------+---------------------+-------------+------------+--------------------+-------------+---------------------+-------------+
|    19607|   91852190|    04-SEP-17|         15.0|                6| 114121913|             16|       Corrigan|            City|      1586|                 N| 

In [12]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')


Enter database password··········


In [15]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-crime.czbxze2tryse.us-east-2.rds.amazonaws.com:5432/database-crime"
config = {"user":"postgres",
          "password": "bootcamp10",
          "driver":"org.postgresql.Driver"}

In [16]:
# Write DataFrame to texas_2020 table in RDS
texas_2020_df.write.jdbc(url=jdbc_url, table='texas_2020', mode=mode, properties=config)

In [17]:
# Write DataFrame to texas_2019 table in RDS
texas_2019_df.write.jdbc(url=jdbc_url, table='texas_2019', mode=mode, properties=config)

In [18]:
# Write DataFrame to texas_2018 table in RDS
texas_2018_df.write.jdbc(url=jdbc_url, table='texas_2018', mode=mode, properties=config)

In [19]:
# Write DataFrame to texas_2017 table in RDS
texas_2017_df.write.jdbc(url=jdbc_url, table='texas_2017', mode=mode, properties=config)